In [172]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import mplfinance as mpf


In [173]:
# เชื่อมต่อกับ Cassandra Docker
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()


In [174]:
# สร้าง keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS stock_data 
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
""")

In [175]:
# ใช้งาน keyspace
session.set_keyspace('stock_data')

In [176]:
# สร้าง table สำหรับเก็บข้อมูลหุ้น
session.execute("""
    CREATE TABLE IF NOT EXISTS stock_price (
        symbol text,
        date date,
        open float,
        high float,
        low float,
        close float,
        volume bigint,
        PRIMARY KEY (symbol, date)
    )
""")

print("✅ Connected and table ready.")

✅ Connected and table ready.


In [177]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
    app_id="ZGCsmM0t5c9VMLRJ",                                 
    app_secret="T5eQ70WLmOWDpVvV7jicJtspuBR2A361nzVGmR8xTF4=", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Hilman-D")
account_info = deri.get_account_info()

account_info 

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [178]:
# Get real-time quote
market_data = investor.MarketData()
stock = market_data.get_quote_symbol("TRUBB")
print(stock)

{'instrumentType': 'STOCK', 'symbol': 'TRUBB', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 0, 'totalNoSideVolume': 0, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': -0.26896, 'pe': 0.0, 'pbv': 0.17, 'percentYield': 0.0, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [179]:
# get candlestick
market_cd = investor.MarketData()
res = market_cd.get_candlestick(symbol="TRUBB", interval="1d",normalized=True,)

In [180]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,0,1681923600,1.97,1.97,1.91,1.92,1393800,2701546.00
1,0,1682010000,1.91,1.94,1.90,1.92,568579,1089752.53
2,0,1682269200,1.91,1.92,1.88,1.89,1497140,2843716.00
3,0,1682355600,1.88,1.90,1.88,1.88,1108055,2088325.97
4,0,1682442000,1.89,1.90,1.87,1.90,270269,509443.04
...,...,...,...,...,...,...,...,...
495,0,1746118800,0.61,0.62,0.60,0.60,360350,217722.50
496,0,1746464400,0.60,0.61,0.60,0.60,438323,263573.26
497,0,1746550800,0.60,0.62,0.60,0.61,741872,449444.92
498,0,1746637200,0.61,0.62,0.60,0.62,203921,124374.80


In [181]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [182]:
# สร้าง DataFrame
df = pd.DataFrame(raw_data)
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# --- วาดกราฟ ---
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))
fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))
fig.update_layout(
    title="📈 กราฟแท่งเทียนหุ้น TRUBB (ล่าสุดจาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)
fig.show()

In [183]:
# --- Insert ลง Cassandra ---
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)

for _, row in df.iterrows():
    session.execute("""
        INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        "TRUBB", row.name.date(), row['open'], row['high'], row['low'], row['close'], int(row['volume'])
    ))

print("✅ ข้อมูลถูก insert ลงใน Cassandra เรียบร้อยแล้ว.")


✅ ข้อมูลถูก insert ลงใน Cassandra เรียบร้อยแล้ว.


In [184]:
# --- อ่านข้อมูลจาก Cassandra มาแสดงผล ---
table_name = "stock_price"  # ✅ ไม่มีเว้นวรรคด้านหน้า
rows = session.execute(f"SELECT * FROM {table_name} LIMIT 10")
df = pd.DataFrame(rows)
df.head(10)

,symbol,date,close,high,low,open,volume
0,TRUBB,1970-01-01,0.62,0.62,0.6,0.62,1417611
